PROYECTO 3

Integrantes:
    - Yosward García Tellez
    - Leonardo Mora Chacón
    - Yurgen Cambronero Mora

In [79]:
# Bilbiotecas a usar
import pandas as pd
import numpy as np
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import trim, lower, regexp_replace
from unidecode import unidecode

In [80]:
def create_spark_session():
    """
    This function builds a Spark Session
    return the main entry of a Spark DataFrame
    """
    spark = SparkSession \
      .builder \
      .appName("Proyecto3") \
      .config("spark.driver.extraClassPath", "postgresql-42.6.0.jar") \
      .config("spark.executor.extraClassPath", "postgresql-42.6.0.jar") \
      .getOrCreate()
    return spark

spark = create_spark_session()

In [81]:
#Separar por columnas el csv y no por comas
df_oij = spark.read.csv("Estadisticas.csv", sep=';', header=True, inferSchema=True)
df_oij.show()

+--------------------+--------------------+----------+-------------------+--------+--------------------+-------------+-------------+--------------+----------+-------------------+--------------+
|              Delito|           SubDelito|     Fecha|               Hora| Victima|          SubVictima|         Edad|       Genero|  Nacionalidad| Provincia|             Canton|      Distrito|
+--------------------+--------------------+----------+-------------------+--------+--------------------+-------------+-------------+--------------+----------+-------------------+--------------+
|DELITOS CONTRA EL...|VIOLACION DE DOMI...| 13/3/2011|18:00:00 - 20:59:59|VIVIENDA|           NO APLICA|Mayor de edad|Mayor de edad|    COSTA RICA|  ALAJUELA|          SAN RAMON|      SAN JUAN|
|DELITOS CONTRA EL...|VIOLACION DE DOMI...|  7/3/2011|00:00:00 - 02:59:59|VIVIENDA|           NO APLICA|Mayor de edad|Mayor de edad|    COSTA RICA|PUNTARENAS|           GARABITO|          JACO|
|DELITOS CONTRA EL...|VIOLACIO

In [82]:
column_names = ['Provincia, canton y distrito', 'Población de 15 años y más', 'Tasa neta de participación',
                          'Tasa_de_ocupacion', 'Tasa de desempleo abierto', 'Porcentaje de población económicamente inactiva',
                          'Relación de dependencia económica', 'Sector Primario', 'Sector Secundario', 'Sector Terciario']

file_inec_excel = pd.read_excel(io="reempleocenso2011-22.xls_2.xls", skiprows=9, header=None, names=column_names, usecols='B:K')

csv_inec = file_inec_excel.to_csv('reempleocenso2011-22.csv', index=False, encoding="UTF-8")

df_inec = spark.read.csv("reempleocenso2011-22.csv",header=True, inferSchema=True)
df_inec.show()

+----------------------------+--------------------------+--------------------------+------------------+-------------------------+-----------------------------------------------+---------------------------------+-------------------+------------------+-----------------+
|Provincia, canton y distrito|Población de 15 años y más|Tasa neta de participación| Tasa_de_ocupacion|Tasa de desempleo abierto|Porcentaje de población económicamente inactiva|Relación de dependencia económica|    Sector Primario| Sector Secundario| Sector Terciario|
+----------------------------+--------------------------+--------------------------+------------------+-------------------------+-----------------------------------------------+---------------------------------+-------------------+------------------+-----------------+
|                        NULL|                      NULL|                      NULL|              NULL|                     NULL|                                           NULL|                

In [83]:
#Eliminar registros donde todas las columnas estan en NULL
def remove_null_rows(df):
    df = df.na.drop("all")
    return df

In [84]:
df_inec = remove_null_rows(df_inec)
df_inec.show()

+----------------------------+--------------------------+--------------------------+------------------+-------------------------+-----------------------------------------------+---------------------------------+-------------------+------------------+-----------------+
|Provincia, canton y distrito|Población de 15 años y más|Tasa neta de participación| Tasa_de_ocupacion|Tasa de desempleo abierto|Porcentaje de población económicamente inactiva|Relación de dependencia económica|    Sector Primario| Sector Secundario| Sector Terciario|
+----------------------------+--------------------------+--------------------------+------------------+-------------------------+-----------------------------------------------+---------------------------------+-------------------+------------------+-----------------+
|                    San José|                 1087315.0|         56.02369138658071|54.076325627807954|       3.4759683101481724|                               43.9763086134193|               1

In [85]:
df_oij = remove_null_rows(df_oij)
df_oij.show()

+--------------------+--------------------+----------+-------------------+--------+--------------------+-------------+-------------+--------------+----------+-------------------+--------------+
|              Delito|           SubDelito|     Fecha|               Hora| Victima|          SubVictima|         Edad|       Genero|  Nacionalidad| Provincia|             Canton|      Distrito|
+--------------------+--------------------+----------+-------------------+--------+--------------------+-------------+-------------+--------------+----------+-------------------+--------------+
|DELITOS CONTRA EL...|VIOLACION DE DOMI...| 13/3/2011|18:00:00 - 20:59:59|VIVIENDA|           NO APLICA|Mayor de edad|Mayor de edad|    COSTA RICA|  ALAJUELA|          SAN RAMON|      SAN JUAN|
|DELITOS CONTRA EL...|VIOLACION DE DOMI...|  7/3/2011|00:00:00 - 02:59:59|VIVIENDA|           NO APLICA|Mayor de edad|Mayor de edad|    COSTA RICA|PUNTARENAS|           GARABITO|          JACO|
|DELITOS CONTRA EL...|VIOLACIO

In [86]:
#Funcion para eliminar espacios en blancos en una columna especifica
def remove_blank_spaces(df, column):
    df = df.withColumn(column, trim(df[column]))
    return df

In [87]:
df_oij = remove_blank_spaces(df_oij, "Distrito")
df_oij.show(20)

+--------------------+--------------------+----------+-------------------+--------+--------------------+-------------+-------------+--------------+----------+-------------------+--------------+
|              Delito|           SubDelito|     Fecha|               Hora| Victima|          SubVictima|         Edad|       Genero|  Nacionalidad| Provincia|             Canton|      Distrito|
+--------------------+--------------------+----------+-------------------+--------+--------------------+-------------+-------------+--------------+----------+-------------------+--------------+
|DELITOS CONTRA EL...|VIOLACION DE DOMI...| 13/3/2011|18:00:00 - 20:59:59|VIVIENDA|           NO APLICA|Mayor de edad|Mayor de edad|    COSTA RICA|  ALAJUELA|          SAN RAMON|      SAN JUAN|
|DELITOS CONTRA EL...|VIOLACION DE DOMI...|  7/3/2011|00:00:00 - 02:59:59|VIVIENDA|           NO APLICA|Mayor de edad|Mayor de edad|    COSTA RICA|PUNTARENAS|           GARABITO|          JACO|
|DELITOS CONTRA EL...|VIOLACIO

In [88]:
df_inec = remove_blank_spaces(df_inec, "Provincia, canton y distrito")
df_inec.show(20)

+----------------------------+--------------------------+--------------------------+------------------+-------------------------+-----------------------------------------------+---------------------------------+-------------------+------------------+-----------------+
|Provincia, canton y distrito|Población de 15 años y más|Tasa neta de participación| Tasa_de_ocupacion|Tasa de desempleo abierto|Porcentaje de población económicamente inactiva|Relación de dependencia económica|    Sector Primario| Sector Secundario| Sector Terciario|
+----------------------------+--------------------------+--------------------------+------------------+-------------------------+-----------------------------------------------+---------------------------------+-------------------+------------------+-----------------+
|                    San José|                 1087315.0|         56.02369138658071|54.076325627807954|       3.4759683101481724|                               43.9763086134193|               1

In [89]:
def set_lower_case(df, column):
    df = df.withColumn(column, lower(df[column]))
    return df


In [90]:
df_oij = set_lower_case(df_oij, "Distrito")
df_oij.show()

+--------------------+--------------------+----------+-------------------+--------+--------------------+-------------+-------------+--------------+----------+-------------------+--------------+
|              Delito|           SubDelito|     Fecha|               Hora| Victima|          SubVictima|         Edad|       Genero|  Nacionalidad| Provincia|             Canton|      Distrito|
+--------------------+--------------------+----------+-------------------+--------+--------------------+-------------+-------------+--------------+----------+-------------------+--------------+
|DELITOS CONTRA EL...|VIOLACION DE DOMI...| 13/3/2011|18:00:00 - 20:59:59|VIVIENDA|           NO APLICA|Mayor de edad|Mayor de edad|    COSTA RICA|  ALAJUELA|          SAN RAMON|      san juan|
|DELITOS CONTRA EL...|VIOLACION DE DOMI...|  7/3/2011|00:00:00 - 02:59:59|VIVIENDA|           NO APLICA|Mayor de edad|Mayor de edad|    COSTA RICA|PUNTARENAS|           GARABITO|          jaco|
|DELITOS CONTRA EL...|VIOLACIO

In [91]:
df_inec = set_lower_case(df_inec, "Provincia, canton y distrito")
df_inec.show()

+----------------------------+--------------------------+--------------------------+------------------+-------------------------+-----------------------------------------------+---------------------------------+-------------------+------------------+-----------------+
|Provincia, canton y distrito|Población de 15 años y más|Tasa neta de participación| Tasa_de_ocupacion|Tasa de desempleo abierto|Porcentaje de población económicamente inactiva|Relación de dependencia económica|    Sector Primario| Sector Secundario| Sector Terciario|
+----------------------------+--------------------------+--------------------------+------------------+-------------------------+-----------------------------------------------+---------------------------------+-------------------+------------------+-----------------+
|                    san josé|                 1087315.0|         56.02369138658071|54.076325627807954|       3.4759683101481724|                               43.9763086134193|               1

In [92]:
def non_matching_districts(df_oij, df_inec):
    oij_districts = df_oij.select("Distrito").distinct()
    inec_districts = df_inec.select("Provincia, canton y distrito").distinct()

    unmatched_district = (oij_districts \
                          .join(inec_districts, oij_districts["Distrito"] == inec_districts["Provincia, canton y distrito"], "left_anti")\
                            .select("Distrito").distinct().collect())
    
    list_unmatched_district = [district["Distrito"] for district in unmatched_district]

    return list_unmatched_district


In [93]:
list_unmatched_districts = non_matching_districts(df_oij, df_inec)
print(list_unmatched_districts)

['rio cuarto', 'san jeronimo', 'isla del coco', 'libano', 'para', 'colon', 'guapiles', 'guacimo', 'puraba', 'cajon', 'belen', 'belen de nosarita', 'granja', 'santa lucia', 'desconocido', 'santo tomas', 'jardin', 'agua buena', 'tarcoles', 'alegria', 'santa maria', 'patarra', 'san sebastian', 'cirri sur', 'cortes', 'san jose de la monta?a', 'palmera', 'pavon', 'canas dulces', 'sanchez', 'llanos de santa lucia', 'juan vinas', 'rio blanco', 'general', 'baru', 'volcan', 'rio azul', 'horquetas', 'bahia ballena', 'rio jimenez', 'ceiba', 'sarchi norte', 'dulce nombre de jesus', 'guacima', 'la legua', 'san jose', 'san ramon', 'aserri', 'dos rios', 'mastate', 'guaycara', 'fortuna', 'sarchi sur', 'cobano', 'san cristobal', 'paramo', 'jesus maria', 'angeles', 'curubande', 'jaco', 'tapezco', 'tres rios', 'jesus', 'san nicolas', 'batan', 'paraiso', 'tilaran', 'cachi', 'espiritu santo', 'merecedes', 'canas', 'mansion', 'rio segundo', 'limon', 'samara', 'tigra', 'mata platano', 'cano negro', 'cairo', 

In [94]:
def count_mismatched(df_oij, df_inec):
    oij_districts = df_oij.select("Distrito")
    inec_districts = df_inec.select("Provincia, canton y distrito")
    
    count = oij_districts.join(inec_districts, oij_districts["Distrito"] == inec_districts["Provincia, canton y distrito"], "left_anti")\
                            .select("Distrito").count()
    return count

print(count_mismatched(df_oij, df_inec))

16089


In [95]:
def remove_special_characters(df, column):

    replacements = {
        'ñ': 'n',
        '\\?': 'n',
        'á': 'a',
        'é': 'e',
        'í': 'i',
        'ó': 'o',
        'ú': 'u'
    }
    for key, value in replacements.items():
        df = df.withColumn(column, regexp_replace(df[column], key, value))
    return df

df_inec = remove_special_characters(df_inec, "Provincia, canton y distrito")
df_oij = remove_special_characters(df_oij, "Distrito")



In [96]:
list_unmatched_districts = non_matching_districts(df_oij, df_inec)
print(list_unmatched_districts)
print(count_mismatched(df_oij, df_inec))

['isla del coco', 'granja', 'desconocido', 'agua buena', 'cortes', 'palmera', 'general', 'horquetas', 'ceiba', 'la legua', 'mastate', 'fortuna', 'tapezco', 'merecedes', 'tigra', 'mata platano', 'cairo', 'asuncion', 'san jose (pizote)', 'macacoma', 'el chirripo', 'monterry', 'los angeles', 'union']
2291


## Guardar Datos de Inec y OIJ en una BD en Postgres

### Configuración
Antes de proceder a realizar el guardado de los datos del INEC y del OIJ, se deben configurar la conexión a Postgresql

- Definición de variables:

In [97]:
POSTGRESQL_URL = "jdbc:postgresql://localhost/"
POSTGRESQL_USER = "postgres"
POSTGRESQL_PASSWORD = "admin"

- Función para realizar el guardado:

In [98]:
def save_spark_df_to_db(df, table_name):
    df \
        .write \
        .format("jdbc") \
        .mode('overwrite') \
        .option("url", POSTGRESQL_URL) \
        .option("user", POSTGRESQL_USER) \
        .option("password", POSTGRESQL_PASSWORD) \
        .option("dbtable", table_name) \
        .save()